<a href="https://colab.research.google.com/github/AreebAhmad-02/Embedding-Models-Finetuning/blob/main/Fine_tuning_Embedding_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install LlamaIndex and its OCR component, llama-parse

In [1]:
!pip install llama-index
!pip install llama-parse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

### API Key Configuration
To use LlamaIndex's OCR capabilities, you need an API key. Obtain your API key by signing up at https://cloud.llamaindex.ai and configure it in your environment variables or directly in your code.

In [3]:
from llama_parse import LlamaParse

parser = LlamaParse(api_key="llx-yfB4Pux4NtvtXBZqU1gzHOXUT5ZfCMCgBX01V9w8UufrWs92")
documents = parser.load_data("/content/data/SalesTax_LegalText.pdf")

Started parsing the file under job_id cac11eca-1423-4b99-9262-0ae917a1b760


In [30]:
from llama_parse import LlamaParse

# Initialize the parser with your API key
parser = LlamaParse(api_key="llx-yfB4Pux4NtvtXBZqU1gzHOXUT5ZfCMCgBX01V9w8UufrWs92")

# Load the PDF file
pdf_path = "/content/data/SalesTax_LegalText.pdf"
documents = parser.load_data(pdf_path)

# Check the total number of documents (pages) loaded
print(f"Total documents loaded: {len(documents)}")

# Iterate through each document (page) and extract the text
for i, doc in enumerate(documents):
    print(f"Page {i+1} Text:")
    print(doc.text)
    print("\n" + "="*50 + "\n")


Started parsing the file under job_id cac11eca-3c5a-4b64-b0e3-c226239a73dc
Total documents loaded: 1
Page 1 Text:
                                      Cityand County of San Francisco                                      Dr. Carlton B. Goodlett PlaceCity Hall
                                                     CertifiedCopy                                       San Francisco, CA94102-4689
                                                        Ordinance
    220536          [ Initiative Ordinance         Business and Tax Regulations Code                    Sales Tax for
                   Transportation Authority
                    Sponsors: Mandelman; Walton, Peskin, Ronen, Melgar; Stefani, Preston, Mar; Safai
                    and Dorsey
                    Ordinance approving a new 2022 Transportation Expenditure Plan for the County
                    Transportation Authority and submitting to the voters at an election to be held onOrdinance amending the Business and Tax Regulat

In [52]:
for doc in documents:
    print(doc.to_dict())


{'id_': '9d9012be-28b1-43b9-8c07-81d3dbe0ddd0', 'embedding': None, 'metadata': {}, 'excluded_embed_metadata_keys': [], 'excluded_llm_metadata_keys': [], 'relationships': {}, 'text': "                                      Cityand County of San Francisco                                      Dr. Carlton B. Goodlett PlaceCity Hall\n                                                     CertifiedCopy                                       San Francisco, CA94102-4689\n                                                        Ordinance\n    220536          [ Initiative Ordinance         Business and Tax Regulations Code                    Sales Tax for\n                   Transportation Authority\n                    Sponsors: Mandelman; Walton, Peskin, Ronen, Melgar; Stefani, Preston, Mar; Safai\n                    and Dorsey\n                    Ordinance approving a new 2022 Transportation Expenditure Plan for the County\n                    Transportation Authority and submitting to the voter

In [44]:
doc_str = str(documents)

In [47]:
len(doc_str.split())

312

In [24]:
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

'Doc ID: 192fafb8-dc8a-4ea3-9a5b-71b6ff7b7678\nText: Cityand County of San Francisco\nDr. Carlton B. Goodlett PlaceCity Hall\nCertifiedCopy                                       San Francisco,\nCA94102-4689\nOrdinance     220536          [ Initiative Ordinan...'

# Using Google Vision API for OCR of PDF

### Using google vision api for Reading scanned pages from PDF

In [9]:
!pip install google-cloud-storage

In [10]:
!pip install --upgrade google-cloud-vision

In [11]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.8.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.6.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.3
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.8
cuda-python==12.2.1
cudf-cu12 @ https://pypi.nvidia.c

In [12]:
import os
# Replace 'YOUR_SERVICE_ACCOUNT_KEY.json' with the actual path to your service account key.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/image-project-123-494614a0d191.json"

#### This code below utilizes Google Cloud Vision API to asynchronously extract text from a PDF document stored in Google Cloud Storage (GCS). The async_detect_document function configures the process by specifying the document type (PDF), creating batches for processing (100 pages each), defining the output location for JSON files containing text locations in GCS, and initiating the asynchronous request.



In [13]:
import json
import re
from google.cloud import vision
from google.cloud import storage

In [14]:
#Supported mime_types are: 'application/pdf' and 'umage/tiff'
def async_detect_document(gcs_source_uri, gcs_destination_uri):

  #Because we are reading from a PDF document, we set this variable
  #to application/pdf.

  #You could also do the same operation with images which you might have
  #pre-processed to make them easier to read.

  mime_type='application/pdf'

  ##Batch size determines how many PDF pages worth of data will go in
  ##each file of text locations

  batch_size = 100

  #We are using a tool which annotates where there is text in an image/pdf
  client = vision.ImageAnnotatorClient()

  feature = vision.Feature(
      type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

  #Here, we tell the Cloud Vision API that our source type is mime_type
  #aka, a PDF, and where that PDF is foudn -gcs source_uri

  gcs_source = vision.GcsSource(uri=gcs_source_uri)
  input_config = vision.InputConfig(
                                                                                gcs_source=gcs_source, mime_type=mime_type)

  #This chunk of code says we will be creating JSON files with
  #106 pages worth of annotation data each, and that we will

  #put these files in gcs_destination_url
  gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)

  output_config = vision.OutputConfig(
      gcs_destination=gcs_destination, batch_size=batch_size)

  #we are making an asynchronous request using the input and output
  #configurations we just set up in the last two chunks

  async_request = vision.AsyncAnnotateFileRequest(
      features=[feature], input_config=input_config,
      output_config=output_config)

  #The operation we will be running will asynchronously batch-annotate files
  #using the client and asyn_request we set up earlier

  operation = client.async_batch_annotate_files(
      requests=[async_request])

  #Now that we've configured how we want to annotate the files,
  #we finaly run the operation.

  #We are setting a timeout so that it doesn't run forever, using up resources
  #if the task we assigned is too big or something goes wrong.

  print('Waiting for the operation to finish.')
  operation.result(timeout=420)


In [25]:
async_detect_document("gs://cloud-vision-read/SalesTax_LegalText.pdf","gs://cloud-vision-read/ocr results/")

Waiting for the operation to finish.


Following text extraction, the **write_to_text function** retrieves the generated JSON output files. It iterates through each file (representing a batch) and extracts the full text content from the first page within that batch. This section can be modified to process text from all pages if needed.

**Key Points:**

The code assumes the input PDF is located at gcs_source_uri and the output JSON files will be written to gcs_destination_uri in GCS.
The batch size of 100 can be adjusted based on your document size and processing needs.
Note: This explanation focuses on the core functionality. Cloud Vision's response contains more detailed information like bounding boxes and confidence scores for the detected text, which you can explore further in the API documentation.

In [36]:
import time

def write_to_text(gcs_destination_uri):
    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()
    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    print("match",match)
    bucket_name = match.group(1)
    print(bucket_name)
    prefix = match.group(2)
    print(prefix)
    bucket = storage_client.get_bucket(bucket_name)
    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
      # Check if the blob is a file (has an extension)
      if '.' not in blob.name:
        blob_list.remove(blob)

    print(len(blob_list))
    print(blob_list)
    for n in range(len(blob_list)):

      # Process the an output file from GCS.
      # Since we specified batch_size=100, the first response contains
      # the first 100 pages of the input file.
      output = blob_list[n]
      print("output----------------", output.name)
      try:
        json_string = output.download_as_string()

        response = json.loads(json_string)
      except ValueError as e:
        print(f"Error decoding JSON: {e}")
        break

      ##llake a file to write the contents of this batch in
      file = open("batch{}.txt".format(str(n)), "w")
      # The actual response for the first page of the input file.
      for m in range(len(response['responses'])):
        first_page_response = response['responses'][m]
        annotation = first_page_response['fullTextAnnotation']
        # Here we print the full text from the first page.
        # The response contains more information:
        # annotation/pages/blocks/paragraphs/words/symbols
        # inctudting confidence scores and bounding boxes
        print('Full text:\n')
        print(annotation['text'])
        file.write(annotation['text'])


In [37]:
write_to_text("gs://cloud-vision-read/ocr results/")

match <re.Match object; span=(0, 35), match='gs://cloud-vision-read/ocr results/'>
cloud-vision-read
ocr results/
Output files:
1
[<Blob: cloud-vision-read, ocr results/output-1-to-57.json, 1718379497163787>]
output---------------- ocr results/output-1-to-57.json
Full text:

CITY AND
UNTY OF SAN FRANCISCO
City and County of San Francisco
Certified Copy
Ordinance
City Hall
1 Dr. Carlton B. Goodlett Place
San Francisco, CA 94102-4689
220536
[Initiative Ordinance - Business and Tax Regulations Code - Sales Tax for
Transportation Authority ]
Sponsors: Mandelman; Walton, Peskin, Ronen, Melgar, Stefani, Preston, Mar, Safai
and Dorsey
Ordinance approving a new 2022 Transportation Expenditure Plan for the County
Transportation Authority and submitting to the voters at an election to be held on
November 8, 2022, an Ordinance amending the Business and Tax Regulations
Code to continue in effect the existing local transactions and use tax at the existing
rate of 0.5% for 30 years to fund transport